# ProbNum Quickstart

ProbNum implements probabilistic numerical methods in Python. Such methods quantify _uncertainty arising from finite computation_ or from _stochastic input_. 

Get started with ProbNum by following these steps.

## Installation

You can install this Python 3 package using `pip` (or `pip3`).
```bash
pip install probnum
```
If you prefer to use the latest version of ProbNum, install directly from GitHub instead.

```bash
pip install git+https://github.com/probabilistic-numerics/probnum.git
```

## Basic Concepts

## Probabilistic Numerical Methods